In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from google.colab import drive


In [2]:

# Mount Google Drive
drive.mount('/content/drive')

# Load the CSV file
df = pd.read_csv('/content/drive/MyDrive/Quora_Project_Training_Set.csv')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Split the data into train and validation sets
train_size = 0.8
train_texts = df['question_text'][:int(train_size*len(df))]
train_labels = df['target'][:int(train_size*len(df))]
val_texts = df['question_text'][int(train_size*len(df)):]
val_labels = df['target'][int(train_size*len(df)):]



In [4]:
# Tokenize the texts
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_texts)

# Convert the texts to sequences
train_seqs = tokenizer.texts_to_sequences(train_texts)
val_seqs = tokenizer.texts_to_sequences(val_texts)

# Pad the sequences with zeros to make them all the same length
max_len = 100
train_seqs = pad_sequences(train_seqs, maxlen=max_len)
val_seqs = pad_sequences(val_seqs, maxlen=max_len)



In [8]:
# Load the pre-trained embeddings
word_index = tokenizer.word_index
embeddings_index = {}
with open('/content/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs


# Create the embedding matrix
embedding_dim = 100
num_words = min(10000, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    if i >= 10000:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector



In [11]:
# Define the model architecture
model = Sequential()
model.add(Embedding(num_words, embedding_dim, input_length=max_len, weights=[embedding_matrix], trainable=False))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
batch_size = 3000
epochs = 2
history = model.fit(train_seqs, train_labels, batch_size=batch_size, epochs=epochs, validation_data=(val_seqs, val_labels))



Epoch 1/2
349/349 [==============================] - 163s 445ms/step - loss: 0.1616 - accuracy: 0.9442 - val_loss: 0.1346 - val_accuracy: 0.9485
Epoch 2/2
349/349 [==============================] - 159s 456ms/step - loss: 0.1328 - accuracy: 0.9497 - val_loss: 0.1277 - val_accuracy: 0.9510


In [12]:

# Evaluate the model on the validation set
score = model.evaluate(val_seqs, val_labels, batch_size=batch_size)
print(f'Validation loss: {score[0]}')
print(f'Validation accuracy: {score[1]}')

# Save the model to a h5 file
model.save('quora_spam_detection_model.h5')

88/88 [==============================] - 6s 66ms/step - loss: 0.1277 - accuracy: 0.9510
Validation loss: 0.12773028016090393
Validation accuracy: 0.9509732723236084
